In [1]:
import pandas as pd
import numpy as np
import random
from sklearn.decomposition import PCA

In [2]:
data = pd.read_csv("C:\\Users\\Lorenzo\\Downloads\\archive (2)\RAW_recipes.csv")
all_tags_unique = data['tags'].apply(eval).explode().unique()

['60-minutes-or-less' 'time-to-make' 'course' 'main-ingredient' 'cuisine'
 'preparation' 'occasion' 'north-american' 'side-dishes' 'vegetables'
 'mexican' 'easy' 'fall' 'holiday-event' 'vegetarian' 'winter' 'dietary'
 'christmas' 'seasonal' 'squash' '30-minutes-or-less' 'breakfast'
 'main-dish' 'pork' 'american' 'oven' 'kid-friendly' 'pizza'
 'northeastern-united-states' 'meat' 'equipment' 'chili'
 'crock-pot-slow-cooker' '4-hours-or-less' 'eggs-dairy' 'potatoes'
 'dinner-party' 'easter' 'cheese' 'stove-top' 'new-years' 'thanksgiving'
 'independence-day' 'st-patricks-day' 'valentines-day' 'inexpensive'
 'brunch' 'superbowl' 'presentation' 'served-hot' 'weeknight' 'canning'
 'condiments-etc' 'heirloom-historical' 'amish-mennonite'
 'number-of-servings' 'technique' '15-minutes-or-less' 'low-protein'
 '5-ingredients-or-less' 'beverages' 'fruit' 'low-sodium' 'shakes'
 'low-calorie' 'low-in-something' 'apples' 'served-cold' '3-steps-or-less'
 'appetizers' 'canadian' 'vegan' 'freezer' 'south

In [22]:
def perform_pca_and_select_tags(df, n_components=10, threshold=0.2, top_n=5):
    all_tags = df['tags'].apply(eval).explode().unique()
    print("All tags: ", len(all_tags), all_tags[:5])

    tag_matrix = pd.DataFrame(0, index=df.index, columns=all_tags)

    for i, tag in enumerate(all_tags):
        tag_matrix.at[i, tag] = 1

    # Apply PCA
    pca_optimal = PCA(n_components=n_components)
    optimal_components = pca_optimal.fit_transform(tag_matrix)
    # Extract explained variance ratio
    explained_variance_ratio = pca_optimal.explained_variance_ratio_

    # Calculate cumulative explained variance
    cumulative_explained_variance = np.cumsum(explained_variance_ratio)

    print("Explained Variance Ratio per Component:")
    print(explained_variance_ratio)

    print("Cumulative Explained Variance:")
    print(cumulative_explained_variance)

    loadings = pd.DataFrame(pca_optimal.components_.T, index=all_tags,
                            columns=[f'PC{i + 1}' for i in range(n_components)])

    top_significant_tags = loadings.apply(
        lambda x: sorted(x, key=lambda _tag: np.abs(loadings.at[_tag, x.name]), reverse=True)[:5])

    print("\nMost Significant Tags for Each Filtered Principal Component (Non-Empty):")
    print(top_significant_tags)

    return top_significant_tags

In [19]:
def filter_recipes(df, selected_tag):
    return df[df['tags'].apply(lambda x: selected_tag in x)]

In [20]:
def simulate_user_choices(df):
    while df.shape[0] > 1:
        print(f"Available recipes = ({df.shape[0]})")
        # Perform PCA and select top tags
        top_significant_tags = perform_pca_and_select_tags(df)
        print(f"TOP SIGNIFICANT = {top_significant_tags}")

        # If no significant tags are found, break the loop
        if top_significant_tags.empty:
            break

        # Flatten the list of significant tags and remove duplicates
        flattened_tags = top_significant_tags.unique()
        print(f"Flatten = {flattened_tags}")
        # If no tags are available, break the loop
        if not flattened_tags:
            break

        # Randomly choose a tag from the top significant tags
        selected_tag = random.choice(flattened_tags)
        print(f"SELECT: {selected_tag}")
        selected_tag.append(selected_tag)

        # Filter recipes based on the selected tag
        df = filter_recipes(df, selected_tag)

    return df

In [23]:
available_recipes = simulate_user_choices(data)

Available recipes = (231637)
All tags:  552 ['60-minutes-or-less' 'time-to-make' 'course' 'main-ingredient' 'cuisine']
Explained Variance Ratio per Component:
[0.0018116 0.0018116 0.0018116 0.0018116 0.0018116 0.0018116 0.0018116
 0.0018116 0.0018116 0.0018116]
Cumulative Explained Variance:
[0.0018116  0.0036232  0.00543481 0.00724641 0.00905801 0.01086961
 0.01268121 0.01449282 0.01630442 0.01811602]


KeyError: -0.04290175094320356